# Random Histogram Forest

In [1]:
import scipy.io as sio
import scipy.stats as sstats
import random 
import numpy as np

In [2]:
# set the number of trees and max height
H = 5
T = 100

### Definition of Node and Leaf classes

In [3]:
# a Node has contains a splitting value, splitting attribute as well as left and right trees
class Node:
    def __init__(self, value, attribute, left, right):
        self.value = value
        self.attribute = attribute
        self.left = left
        self.right = right

    # prints the data and splitting information in order of left branch, parent node, right branch
    def printNode(self, level):
        tabs = ""
        for i in range(0, level):
            tabs += "\t"
        # Leaf case
        if (self.left is None and self.right is None):
            print(tabs + str(self.data))
        else:
            # taking into account splits that have all instances on one side
            if self.left is not None:
                Node.printNode(self.left, level+1)
            print(tabs + "a" + str(self.attribute) + " < " + str(self.value))
            if self.right is not None:
                Node.printNode(self.right, level+1)

# a Leaf is a Node that has all parameters set to None except for data which contains its instances
class Leaf(Node):
    def __init__(self, data):
        super().__init__(value=None, attribute=None, left=None, right=None)
        self.data = data

### Splitting functions

In [4]:
# use kurtosis sum to get best attribute for the split
# returns attribute, its column and a random split value
def get_attribute(X):
    # attribute selected according to kurtosis
    ks = kurtosis_sum(X, X.shape[1]-1)
    # ks may not be included depending on rounding
    r = random.uniform(0, ks)
        
    for a in range(0, X.shape[1]):
        if kurtosis_sum(X, a) > r:
            a_col = X[:, a]
                        
            # ensures that the split will be proper (no split on extremes)
            a_val = np.amin(a_col)

            while a_val == np.amin(a_col) or a_val == np.max(a_col):
                a_val = random.uniform(np.amin(a_col), np.amax(a_col))

            return a, a_col, a_val
        
# sum of log(Kurtosis(X[a] + 1)) of attributes 0 to d inclusive
def kurtosis_sum(X, d):
    sum = 0
    
    # loop over the transpose matrix in order to analyze by column
    for a in range(0, d+1):
        # + 4 since the scipy function for kurtosis subtracts 3 so +1 +3 = 4
        sum += np.log(sstats.stats.kurtosis(X[:,a])+4)
        
    return sum

### Core algorithms

In [5]:
# construction of a random histogram tree
def rht(X, nd, h):
    # last condition checks if all instances are the same --> split not possible
    if nd >= h or X.shape[0] == 1 or np.unique(X, axis=0).size == 1:
        # returns instances without duplicates
        return Leaf(np.unique(X, axis=0))
    else:
        a, a_col, a_val = get_attribute(X)
             
        Xl = X[X[:, a] < a_val]
        Xr = X[X[:, a] >= a_val]

        Xl = rht(Xl, nd + 1, h)
        Xr = rht(Xr, nd + 1, h)

        return Node(a_val, a, Xl, Xr)

# construction of a random histogram forest
def rhf(X, t, nd, h):
    # create an empty forest
    rhf = np.empty([t], dtype=object)

    # append t random histogram trees
    for i in range(t):
        rhf[i] = rht(X, nd, h)

    return rhf

### Anomaly scoring

In [6]:
def anomaly_score(rhf, n, x):
    sum = 0
    
    for i in range(0, rhf.size):
        # number of distinct instances in the leaf of the given instance
        leaf_size = find_instance(rhf[i], x).data.size
        p = leaf_size / n
        sum += np.log(1 / p)
        
    return sum

# returns Leaf of instance in a RHT
def find_instance(rht, x):
    if rht.left != None or rht.right != None:
        # not leaf
        a = rht.attribute
        a_val = rht.value
        
        if (x[a] < a_val):
            return find_instance(rht.left, x)
        else: 
            return find_instance(rht.right, x)
    else:
        # leaf
        if x in rht.data:
            return rht

### Unit tests

### Testing

In [7]:
mat_contents = sio.loadmat("satellite.mat")

dataset = mat_contents['X'] 
labels = mat_contents['y']


test_rhf = rhf(X=dataset, t=T, nd=0, h=H)





'''
scores = np.array([dataset.shape[0], 2])
for i, x in enumerate(dataset):
    score = anomaly_score(test_rhf, dataset.size, x)
    np.append(scores, [score, labels[i]])
    print("score = " + str(score) + " | anomaly? " + str(labels[i]))
'''

    
    


score = 280.2645721725584 | anomaly? [0]
score = 138.90602236602018 | anomaly? [0]
score = 122.39232180660514 | anomaly? [0]
score = 119.83514299871942 | anomaly? [0]
score = 121.67324555852883 | anomaly? [0]
score = 127.28402275340991 | anomaly? [0]
score = 118.57249534010485 | anomaly? [0]
score = 116.78727425613778 | anomaly? [0]
score = 105.18779964875732 | anomaly? [1]
score = 108.62945236802892 | anomaly? [1]
score = 104.19939288827374 | anomaly? [1]
score = 105.24492709841566 | anomaly? [1]
score = 112.15174583285607 | anomaly? [1]
score = 111.26557414043039 | anomaly? [1]
score = 107.19228337347971 | anomaly? [1]
score = 100.36860043132319 | anomaly? [1]
score = 101.39832549276706 | anomaly? [1]
score = 108.38906152134541 | anomaly? [1]
score = 122.71576001869005 | anomaly? [0]
score = 128.24871592535706 | anomaly? [0]
score = 111.2692550670693 | anomaly? [0]
score = 120.22002299215306 | anomaly? [0]
score = 137.8946539323094 | anomaly? [0]
score = 155.41056306793973 | anomaly?

score = 144.90216428402422 | anomaly? [0]
score = 134.263500265945 | anomaly? [0]
score = 128.52893160096158 | anomaly? [0]
score = 140.7021002938276 | anomaly? [0]
score = 136.4650807451707 | anomaly? [0]
score = 135.08765387093152 | anomaly? [0]
score = 131.6860562514306 | anomaly? [0]
score = 129.16171165206683 | anomaly? [0]
score = 131.73212717425082 | anomaly? [0]
score = 129.42897342623291 | anomaly? [0]
score = 127.87131927316568 | anomaly? [0]
score = 129.64801195377262 | anomaly? [0]
score = 123.80423544182389 | anomaly? [0]
score = 116.59975663573793 | anomaly? [0]
score = 123.0863652441366 | anomaly? [0]
score = 122.11128995913596 | anomaly? [0]
score = 129.1663124818194 | anomaly? [0]
score = 111.40676633948176 | anomaly? [0]
score = 117.81848172198391 | anomaly? [0]
score = 118.43558512334312 | anomaly? [0]
score = 116.85889114377491 | anomaly? [0]
score = 112.8963706829958 | anomaly? [0]
score = 121.52531701287128 | anomaly? [0]
score = 124.27272019438682 | anomaly? [0]


score = 128.56860019541637 | anomaly? [0]
score = 139.1666088632676 | anomaly? [0]
score = 130.79989437983474 | anomaly? [0]
score = 124.08846009178559 | anomaly? [0]
score = 126.55023203519762 | anomaly? [0]
score = 128.28768473542502 | anomaly? [0]
score = 134.05518568097756 | anomaly? [0]
score = 131.10104037312846 | anomaly? [0]
score = 141.48330429585843 | anomaly? [0]
score = 117.712229190196 | anomaly? [0]
score = 124.62265802006463 | anomaly? [0]
score = 127.68137977411126 | anomaly? [0]
score = 123.92966340145828 | anomaly? [0]
score = 125.92152058911816 | anomaly? [0]
score = 123.43313577914641 | anomaly? [0]
score = 131.36315022001227 | anomaly? [0]
score = 148.29934705885643 | anomaly? [0]
score = 140.14026157554406 | anomaly? [0]
score = 126.36959864044523 | anomaly? [0]
score = 123.55417936418144 | anomaly? [0]
score = 118.56047580271914 | anomaly? [0]
score = 119.44508390024481 | anomaly? [0]
score = 137.89078417378525 | anomaly? [0]
score = 121.26573201689578 | anomaly?

score = 291.7203490524771 | anomaly? [0]
score = 217.27301187117374 | anomaly? [0]
score = 164.04829061939685 | anomaly? [0]
score = 141.46173213125357 | anomaly? [0]
score = 162.11832032818427 | anomaly? [0]
score = 161.5884273217431 | anomaly? [0]
score = 144.47129050742012 | anomaly? [0]
score = 129.52917481685105 | anomaly? [0]
score = 140.52043114300716 | anomaly? [0]
score = 191.7288892247805 | anomaly? [1]
score = 270.4042392874689 | anomaly? [1]
score = 285.66162365887465 | anomaly? [1]
score = 280.146923279728 | anomaly? [1]
score = 292.79518423668765 | anomaly? [1]
score = 289.26786353092217 | anomaly? [1]
score = 290.34400819913395 | anomaly? [1]
score = 288.0573239802087 | anomaly? [1]
score = 286.4713800008302 | anomaly? [1]
score = 280.0518980700303 | anomaly? [1]
score = 274.02557743071173 | anomaly? [1]
score = 280.665240836029 | anomaly? [1]
score = 296.78027581153856 | anomaly? [1]
score = 329.87967616756623 | anomaly? [1]
score = 336.8592198648061 | anomaly? [1]
scor

score = 149.21941865685602 | anomaly? [0]
score = 149.7863726186248 | anomaly? [0]
score = 251.1948359968105 | anomaly? [1]
score = 248.18302802359605 | anomaly? [1]
score = 224.45113044153575 | anomaly? [1]
score = 221.19167705358277 | anomaly? [1]
score = 203.8276024148265 | anomaly? [1]
score = 126.16170979808993 | anomaly? [0]
score = 115.678165030033 | anomaly? [0]
score = 104.32565258174373 | anomaly? [0]
score = 142.97772531747947 | anomaly? [0]
score = 136.41767077787603 | anomaly? [0]
score = 141.99773840873567 | anomaly? [0]
score = 143.18561247299075 | anomaly? [0]
score = 142.7637485337375 | anomaly? [0]
score = 145.5986397076333 | anomaly? [0]
score = 177.08904895823974 | anomaly? [0]
score = 171.17325416355354 | anomaly? [0]
score = 172.86821794983854 | anomaly? [0]
score = 133.80163956292546 | anomaly? [0]
score = 120.80278788305263 | anomaly? [1]
score = 126.05833906818525 | anomaly? [1]
score = 119.88290041362569 | anomaly? [1]
score = 234.13604692902592 | anomaly? [1]

...

score = 175.93573083318682 | anomaly? [0]
score = 182.85256756578042 | anomaly? [0]
score = 158.47144058334285 | anomaly? [0]
score = 137.86565720701344 | anomaly? [0]
score = 142.0664384753704 | anomaly? [0]
score = 106.00277914501116 | anomaly? [1]
score = 114.83325941565646 | anomaly? [1]
score = 126.85642803885695 | anomaly? [0]
score = 114.17940354476886 | anomaly? [1]
score = 120.16887524394362 | anomaly? [1]
score = 149.88359297404494 | anomaly? [1]
score = 120.1722303885938 | anomaly? [0]
score = 118.62398285883093 | anomaly? [0]
score = 113.61870994041405 | anomaly? [0]
score = 113.60056204917133 | anomaly? [0]
score = 149.5816791829246 | anomaly? [1]
score = 145.40588661594234 | anomaly? [1]
score = 146.6971667784089 | anomaly? [1]
score = 141.21161151967368 | anomaly? [0]
score = 156.79478404251566 | anomaly? [0]
score = 169.9150980473749 | anomaly? [0]
score = 165.51136259566306 | anomaly? [0]
score = 172.8348955652832 | anomaly? [0]
score = 122.51679391882594 | anomaly? [0

score = 110.4696575087919 | anomaly? [0]
score = 108.43771634248586 | anomaly? [1]
score = 116.69910928892543 | anomaly? [1]
score = 145.08418632749908 | anomaly? [1]
score = 113.57655188013096 | anomaly? [0]
score = 115.83771884485397 | anomaly? [0]
score = 119.06427806969228 | anomaly? [0]
score = 115.9181144027262 | anomaly? [0]
score = 126.24655815569692 | anomaly? [0]
score = 190.93836163167748 | anomaly? [1]
score = 163.83297378024542 | anomaly? [1]
score = 160.92043972695006 | anomaly? [1]
score = 206.98443743310983 | anomaly? [1]
score = 140.29705579129921 | anomaly? [0]
score = 121.30736094941729 | anomaly? [0]
score = 119.82688320329828 | anomaly? [0]
score = 121.59983509326491 | anomaly? [0]
score = 139.77936876035483 | anomaly? [0]
score = 152.1566204149473 | anomaly? [0]
score = 154.40075832516843 | anomaly? [0]
score = 144.38833910600812 | anomaly? [0]
score = 112.66605532333766 | anomaly? [1]
score = 117.9200097429985 | anomaly? [1]
score = 113.67673215855108 | anomaly? 

score = 116.10228644956027 | anomaly? [0]
score = 124.19877568226747 | anomaly? [0]
score = 128.07892961404576 | anomaly? [0]
score = 117.98167365744914 | anomaly? [0]
score = 113.5398961132464 | anomaly? [0]
score = 107.98097507595037 | anomaly? [0]
score = 160.74826677376726 | anomaly? [0]
score = 164.21899102383833 | anomaly? [0]
score = 148.19485989685396 | anomaly? [0]
score = 112.3843176893406 | anomaly? [0]
score = 123.44048414823897 | anomaly? [0]
score = 127.07926484254041 | anomaly? [0]
score = 124.52285692040167 | anomaly? [0]
score = 117.46737579593471 | anomaly? [0]
score = 108.992077067757 | anomaly? [0]
score = 114.21151880810217 | anomaly? [0]
score = 124.85333318836207 | anomaly? [0]
score = 126.62776130902002 | anomaly? [0]
score = 131.09818066285837 | anomaly? [0]
score = 121.35846147403069 | anomaly? [0]
score = 132.31733178761505 | anomaly? [0]
score = 145.27457626946608 | anomaly? [0]
score = 145.4006585050319 | anomaly? [0]
score = 141.624685579769 | anomaly? [0]

score = 143.14887035532684 | anomaly? [1]
score = 122.38131382463003 | anomaly? [0]
score = 124.37535077101175 | anomaly? [0]
score = 121.54180737420857 | anomaly? [0]
score = 120.10648843508773 | anomaly? [0]
score = 126.43150736642764 | anomaly? [0]
score = 143.46083822697048 | anomaly? [0]
score = 117.4509235155084 | anomaly? [0]
score = 117.26001561857137 | anomaly? [0]
score = 113.87730798635634 | anomaly? [0]
score = 120.22101227769735 | anomaly? [0]
score = 119.58613321645656 | anomaly? [0]
score = 110.35454766651208 | anomaly? [0]
score = 122.71661521486608 | anomaly? [0]
score = 145.02330746951236 | anomaly? [0]
score = 143.50834116465919 | anomaly? [0]
score = 141.994677743428 | anomaly? [0]
score = 143.00600206559815 | anomaly? [0]
score = 142.7007354217764 | anomaly? [0]
score = 139.68062697830862 | anomaly? [0]
score = 113.00614156373673 | anomaly? [0]
score = 104.88972725138433 | anomaly? [0]
score = 99.71939464191632 | anomaly? [0]
score = 117.16116116106636 | anomaly? [

score = 145.64442886986595 | anomaly? [0]
score = 142.80949946210262 | anomaly? [0]
score = 141.49092073057548 | anomaly? [0]
score = 113.26712816559356 | anomaly? [0]
score = 110.99462026959635 | anomaly? [0]
score = 107.83196692277728 | anomaly? [0]
score = 99.83994111068235 | anomaly? [0]
score = 109.03352682084967 | anomaly? [0]
score = 115.17884560805656 | anomaly? [0]
score = 124.35859411791463 | anomaly? [0]
score = 110.0051389177635 | anomaly? [0]
score = 126.44334384053941 | anomaly? [0]
score = 119.87009684631833 | anomaly? [0]
score = 143.25351877447957 | anomaly? [0]
score = 140.6163472628019 | anomaly? [0]
score = 111.08840903264958 | anomaly? [0]
score = 112.46937223775367 | anomaly? [0]
score = 118.21618288138637 | anomaly? [0]
score = 153.14156114539998 | anomaly? [1]
score = 163.80387029104767 | anomaly? [1]
score = 142.72325358228423 | anomaly? [1]
score = 162.6388359360674 | anomaly? [0]
score = 109.97296326755234 | anomaly? [0]
score = 115.12619946837914 | anomaly? 

### Plotting